# Analyse de données avec les Spark DataFrames

Votre mission : Répondre aux questions suivantes sur le jeu de données Titanic à l'aide des DataFrames Spark.

Voici quelques fonctions utiles :

- `dataFrame.select('colonne1', 'colonne2')` sélectionne les colonnes `colonne1` et `colonne2` du DataFrame `dataFrame`
- `dataFrame.orderBy('colonne')` trie les lignes du DataFrame `dataFrame` par la colonne `colonne`
- `dataFrame.groupBy('colonne')` groupe les lignes du DataFrame `dataFrame` par la colonne `colonne`
- `dataFrame.count()` renvoie le nombre de lignes du DataFrame `dataFrame`
- `dataFrame.mean()` renvoie la moyenne des valeurs numériques du DataFrame `dataFrame`

Grâce à la force de la programmation fonctionnelle, il est possible de combiner ces fonctions pour effectuer des analyses complexes en quelques lignes de code :

- `dataFrame.groupBy('colonne').count()` compte le nombre de lignes par valeur distincte de la colonne `colonne`
- `dataFrame.groupBy('colonne').count().orderBy('count', ascending=False).show()` compte le nombre de lignes par valeur distincte de la colonne `colonne` et trie les résultats par ordre décroissant
- etc...

Bonne chance!

## Initialisation du notebook

In [1]:
print("Hello, World from Scala kernel!")

Intitializing Scala interpreter ...

Spark Web UI available at http://23d3897efa1d:4040
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1703629762686)
SparkSession available as 'spark'


Hello, World from Scala kernel!

In [34]:
val titanic = spark
                .read
                .format("csv")
                .option("header", "true")
                .option("inferSchema", "true")
                .load("../data/transformed_titanic.csv")

titanic: org.apache.spark.sql.DataFrame = [PassengerId: int, Survived: int ... 13 more fields]


In [65]:
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- FamilySize: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- AgeCategory: string (nullable = true)



## Question 1 : Quel est le nombre de passagers sur le Titanic ?

In [22]:
print(titanic.count() + "passagers")

Le nombre de passagers est de : 889

## Question 2 : Quels sont les 5 passagers du Titanic les plus âgés ?

In [38]:
titanic.select("Name", "Age").orderBy($"Age".desc).show(5)

Les 5 passagers du Titanic les plus âgés sont :
+--------------------+----+
|                Name| Age|
+--------------------+----+
|Barkworth, Mr. Al...|80.0|
| Svensson, Mr. Johan|74.0|
|Goldschmidt, Mr. ...|71.0|
|Artagaveytia, Mr....|71.0|
|Connors, Mr. Patrick|70.5|
+--------------------+----+
only showing top 5 rows



## Question 3 : Quels sont les 5 passagers du Titanic qui ont payé le moins cher leur ticket ?

In [40]:
titanic.select("Name", "Fare").orderBy("Fare").show(5)

+--------------------+----+
|                Name|Fare|
+--------------------+----+
|Reuchlin, Jonkhee...| 0.0|
|Watson, Mr. Ennis...| 0.0|
|Frost, Mr. Anthon...| 0.0|
|Andrews, Mr. Thom...| 0.0|
|Cunningham, Mr. A...| 0.0|
+--------------------+----+
only showing top 5 rows



## Question 4 : Quel est l'âge moyen des passagers du Titanic ?


In [57]:
titanic.select(avg("Age")).show()

+-----------------+
|         avg(Age)|
+-----------------+
|29.69589060308555|
+-----------------+



## Question 5 :  Quel est le prix que 50% des passagers ont payé pour accéder au Titanic ?


In [75]:
titanic.select(median($"Age")).show()

+-----------+
|median(Age)|
+-----------+
|       28.0|
+-----------+



## Question 6 : Combien de passagers par classe ont survécu ?

In [85]:
titanic.groupBy("Pclass").count().show()

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  489|
|     2|  184|
+------+-----+



## Question 7 : Quelle est la somme totale des tarifs payés par classe ?

In [86]:
titanic.groupBy("Pclass").sum("Fare").show()

+------+------------------+
|Pclass|         sum(Fare)|
+------+------------------+
|     1|18177.412499999988|
|     3|6691.0868000000055|
|     2|3801.8417000000004|
+------+------------------+



## Question 8 : Quel est le nombre de survivants sur l'ensemble du Titanic ?

In [74]:
titanic.filter("Survived == 1").groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  341|
+--------+-----+



## Question 9 : Quel est le pourcentage de passagers qui ont embarqué à chaque port ?

In [83]:
titanic.groupBy("Embarked").agg(count("Embarked") / titanic.count() * 100).show()

+--------+-------------------------------+
|Embarked|((count(Embarked) / 889) * 100)|
+--------+-------------------------------+
|       Q|              8.661417322834646|
|    NULL|                            0.0|
|       C|             18.785151856017997|
|       S|              72.32845894263217|
+--------+-------------------------------+



## Question 10 : Quel est le pourcentage de passagers parmi les femmes ayant survécu ?

In [96]:
titanic.groupBy("Sex").agg(count("Survived") / titanic.count() * 100).filter("Sex == 'female'").show()

+------+-------------------------------+
|   Sex|((count(Survived) / 889) * 100)|
+------+-------------------------------+
|female|              35.32058492688414|
+------+-------------------------------+

